In [23]:
from numpy import *
import time
import sys
import scipy.io

print("enter file name: ", end="")
fil = input()  

fil= fil +".mat"
print("Reading data from file '" + fil + "'")

if fil==None:
    print('File '+repr(fil)+' does not exist')
    exit()

print('Datafile: '+repr(fil))
mat=scipy.io.loadmat(fil,squeeze_me=True)

c=mat['c']
A=mat['A']
b=mat['b']
bix=asarray(mat['bix'],dtype=int)
zcheat=mat['zcheat']
if 'xcheat' in mat:
    xcheat=mat['xcheat']
else:
    xcheat=0

bix=bix-1
[m,n] = shape(A)
print('Rows: '+repr(m)+' cols: '+repr(n))

print('c:',c)
print('b:',b)
print('A:',A)
print('bix:',bix)
print('xcheat:',xcheat)
print('zcheat:',zcheat)



enter file name: Reading data from file 'lab1-cloetta.mat'
Datafile: 'lab1-cloetta.mat'
Rows: 3 cols: 5
c: [-2 -1  0  0  0]
b: [ 1.    0.75 10.  ]
A: [[ 1  1  1  0  0]
 [ 1  0  0  1  0]
 [ 8 20  0  0  1]]
bix: [2 3 4]
xcheat: [0.75 0.2  0.05 0.   0.  ]
zcheat: -1.7


In [150]:
from ctypes import sizeof
import numpy as np
import time
import sys

rc_min = np.nanmin(c)


# npzfile= fil
# c=npzfile['c']
# b=npzfile['b']
# A=npzfile['A']
# bix=npzfile['bix']
# zcheat=npzfile['zcheat']
# xcheat=npzfile['xcheat']


t1=time.time()

[m,n] = np.shape(A)
print( 'Rows: '+repr(m)+' cols: '+repr(n))

# Create nix
nix=np.setdiff1d(range(n), bix) 

B  = A[:, bix]
N  = A[:, nix]
cB = c[bix]
cN = c[nix]
# Start iterations
iter = 0
while iter >= 0:
    iter+=1
    inverseB = np.linalg.inv(B)
    inkvar = int(np.where(cN==np.nanmin(cN))[0])
    print("inkvar", inkvar)
    rc_min = np.nanmin(cN)

    y = np.transpose(np.matmul(np.transpose(cB), inverseB))

    # calc right-hand-sides and reduced costs
    a= b/np.transpose(N)[inkvar] #inkommande var kol genom b (bhat), nu bestäms utgående av det minsta

   

    
    cN = np.subtract(cN, np.matmul(np.transpose(N),y))
    print(cN)

    utgvarPlats = int(np.where(a == np.nanmin(a))[0])
    utgvar = bix[utgvarPlats]
    

    # --------


    # calc most negative reduced cost, rc_min,
    # and index for entering variable, inkvar
    # --------
    rc_min = 1; 


    if rc_min >= -1.0E-12:
        print ('Ready')
        iter=-1
 
        # construct solution, x, and check it
        # --------

        x = np.full([np.size(c)], 0)
        a.resize(x.shape)
        x = a+x
    

    
        z= np.matmul(np.transpose(b),y)
        print(z)


        diffx = np.linalg.norm(x-xcheat)
        diffz = z-zcheat
        print ('xdiff: '+repr(diffx))
        print ('zdiff: '+repr(diffz))
    else:
        # calc entering column, a
        # -------- 
        a= np.matmul(inverseB, a)
        print(a)
        if max(a) <= 0 :
            # unbounded solution
            print('Unbounded solution!')
            iter=-1
        else:
            # calc leaving var, utgvar
            # --------
            utgvar = int(np.where(a == np.nanmin(a))[0])


            print(' Iter: '+repr(iter)+' z: '+repr(z)+' rc: '+repr(rc_min)+' ink: '+repr(inkvar+1)+' utg: '+repr(utgvar+1))

            # make new partition
            # -------
            newB = np.array(B)
            print(np.transpose(B)[utgvarPlats])

            np.transpose(newB)[utgvarPlats] =  np.transpose(N)[inkvar]
            np.transpose(N)[inkvar] = np.transpose(B)[utgvarPlats]
            B = newB
            

            bix[utgvarPlats] = inkvar
            print(bix)



elapsed = time.time() - t1
print('Elapsed time: '+repr(elapsed))

Rows: 3 cols: 5
inkvar 0
[-2. -1.]
Ready
0.0
xdiff: 1.3435028842544403
zdiff: 1.7
Elapsed time: 0.00199127197265625
